Combine embeddings from different integration methods into single adata instance apropritae for scIB (data preparation for integration is in integration_gsPanglaoReactome.ipynb).

In [3]:
import scanpy as sc
import pandas as pd

In [4]:
path_data='/storage/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/qtr/integrated/gsCellType_query/querySTZ_rmNodElimination/mo/'
params='_a_p0.0r1.0-akl_0.1-aklq_0.1-ra_0-uh_0-b_study_sample-sd_False-dp_0.01-lr_0.001-hls_830.830.830-es_1-nh_10000-ne_500-dll_softplus-ule_False-mg_200-mig_3-aea_100-aeaq_None-wd_0.0'

In [5]:
# Data for integration (raw counts)
adata=sc.read(path_data+'adata_integration_RefQueryTraining'+params+'.h5ad')

Only considering the two last: ['.0', '.h5ad'].
Only considering the two last: ['.0', '.h5ad'].


In [6]:
adata

AnnData object with n_obs × n_vars = 82077 × 6572
    obs: 'leiden_r1_normscl', 'design', 'age', 'sex', 'sample', 'n_counts', 'n_genes', 'mt_frac', 'study', 'study_sample', 'cell_type', 'ref_query'
    var: 'gene_symbol'
    obsm: 'X_umap_normscl'

### For plotting & scIB

Add integrated embeddings

In [12]:
# Integrated embeddings from Sergey's data
adata_int=sc.read(
    '/storage/groups/ml01/workspace/sergei.rybakov/adata_integration_RefQueryTraining.h5ad')
for emb in ['X_scvi','X_seurat','X_symphony']:
    adata.obsm[emb]=adata_int[adata.obs_names,:].obsm[emb]

In [13]:
# Integrated embedding from qtr
adata.obsm['X_qtr']=pd.read_table(path_data+'latent_refqueryTraining'+params+'.tsv',
                                       index_col=0
                                      ).loc[adata.obs_names,:].values

In [14]:
adata

AnnData object with n_obs × n_vars = 82077 × 6572
    obs: 'leiden_r1_normscl', 'design', 'age', 'sex', 'sample', 'n_counts', 'n_genes', 'mt_frac', 'study', 'study_sample', 'cell_type', 'ref_query'
    var: 'gene_symbol'
    obsm: 'X_umap_normscl', 'X_scvi', 'X_seurat', 'X_symphony', 'X_qtr'

Save

In [15]:
# Save data with all embeddings to original location
adata.write(path_data+'adata_integration_RefQueryTraining'+params+'.h5ad')

### For scIB

Remove non-healthy query as it is not expected to integrate

In [16]:
# Remove non-healthy query as it is not sure whether it should integrate or not - 
# e.g. expected to be different
adata_norm_sub=adata[~((adata.obs.study=='STZ').values & (adata.obs.design!='control').values),:]
print(adata_norm_sub.shape)
print(adata_norm_sub.obs.query('study=="STZ"').design.unique())

(52270, 6572)
['control']
Categories (21, object): ['A1_r1', 'A1_r2', 'A1_r3', 'A10_r1', ..., 'control', 'mGFP', 'mRFP', 'mTmG']


Normalise

In [17]:
# Normalised data
sc.pp.normalize_total(adata_norm_sub, target_sum=1e6, exclude_highly_expressed=True)
sc.pp.log1p(adata_norm_sub)

/home/icb/karin.hrovatin/miniconda3/envs/rpy2_3/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:155: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [18]:
adata_norm_sub

AnnData object with n_obs × n_vars = 52270 × 6572
    obs: 'leiden_r1_normscl', 'design', 'age', 'sex', 'sample', 'n_counts', 'n_genes', 'mt_frac', 'study', 'study_sample', 'cell_type', 'ref_query'
    var: 'gene_symbol'
    uns: 'log1p'
    obsm: 'X_umap_normscl', 'X_scvi', 'X_seurat', 'X_symphony', 'X_qtr'

Save

In [19]:
# Save for scIB
adata_norm_sub.write(path_data+'adata_norm_integrated_forScib'+params+'.h5ad')

In [20]:
path_data+'adata_norm_integrated_forScib'+params+'.h5ad'

'/storage/groups/ml01/workspace/karin.hrovatin/data/pancreas/scRNA/qtr/integrated/gsCellType_query/querySTZ_rmNodElimination/mo/adata_norm_integrated_forScib_a_p0.0r1.0-akl_0.1-aklq_0.1-ra_0-uh_0-b_study_sample-sd_False-dp_0.01-lr_0.001-hls_830.830.830-es_1-nh_10000-ne_500-dll_softplus-ule_False-mg_200-mig_3-aea_100-aeaq_None-wd_0.0.h5ad'